In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

In [6]:
# Ruta al dataset crudo
RAW_DATASET_PATH = "../data/raw/ddos_final2.csv"

# Ruta donde se guardará el dataset limpio
CLEANED_DATASET_PATH = "../data/processed/ddos_findef.csv"

In [7]:
print("Configurando rutas de entrada y salida...")
output_dir = Path(CLEANED_DATASET_PATH).parent
output_dir.mkdir(parents=True, exist_ok=True)
print(f"   Directorio de salida: {output_dir}")

Configurando rutas de entrada y salida...
   Directorio de salida: ../data/processed


Carga de datos

In [8]:
print(f"\nCargando dataset desde: {RAW_DATASET_PATH}")
try:
    df = pd.read_csv(RAW_DATASET_PATH)
    print("Dataset cargado exitosamente.")
    print("Dimensiones iniciales:", df.shape)
    # print("\nPrimeras 5 filas del dataset crudo:")
    # print(df.head())
except FileNotFoundError:
    print(f"❌ Error: El archivo {RAW_DATASET_PATH} no fue encontrado. Verifica la ruta.")
    exit() # Detener la ejecución si no se puede cargar el archivo
except Exception as e:
    print(f"❌ Error inesperado al cargar el dataset: {e}")
    exit()


Cargando dataset desde: ../data/raw/ddos_final2.csv
Dataset cargado exitosamente.
Dimensiones iniciales: (2660377, 20)


Separación de Características (X) y Objetivo (y)

In [9]:
print("\nSeparando características y variable objetivo...")
# Crear variable objetivo binaria si 'Attack_Label' existe
if 'Attack_Label' in df.columns:
    df["y"] = (df["Attack_Label"].astype(str) != "BENIGN").astype(int)
    print("   Variable objetivo 'y' creada.")
    feature_cols = [c for c in df.columns if c not in ["Attack_Label", "y"]]
    y = df["y"].copy()
else:
    print("   Advertencia: Columna 'Attack_Label' no encontrada. Se procesarán todas las columnas como características.")
    feature_cols = df.columns.tolist()
    y = None # No hay variable objetivo definida explícitamente

# Seleccionar solo las características
X = df[feature_cols].copy()
print(f"   {len(feature_cols)} columnas seleccionadas como características (X).")


Separando características y variable objetivo...
   Variable objetivo 'y' creada.
   19 columnas seleccionadas como características (X).


Saneamiento y Limpieza de Datos (X)

In [10]:
# 1. Reemplazar valores infinitos (inf, -inf) por NaN
print("Paso 1: Reemplazando valores infinitos (inf) por NaN...")
numeric_cols_only = X.select_dtypes(include=np.number).columns # Select only numeric columns for inf check
initial_inf_count = np.isinf(X[numeric_cols_only].values).sum() # Use .values for direct numpy operation
X = X.replace([np.inf, -np.inf], np.nan)
final_inf_count = np.isinf(X[numeric_cols_only].values).sum()
print(f"   {initial_inf_count} valores infinitos reemplazados por NaN. Infinitos restantes: {final_inf_count}")

Paso 1: Reemplazando valores infinitos (inf) por NaN...
   4126 valores infinitos reemplazados por NaN. Infinitos restantes: 0


In [11]:
# 2. Imputación Específica para columnas problemáticas (Flow Bytes/s, Flow Packets/s)
print("\nPaso 2: Imputando NaNs específicos con 0...")
problem_cols = ['Flow Bytes/s', 'Flow Packets/s']
nan_before_step2 = {}
nan_after_step2 = {}
for col in problem_cols:
    if col in X.columns:
        nan_before_step2[col] = X[col].isnull().sum()
        if X[col].isnull().any(): # Solo rellenar si hay NaNs
             print(f"   Rellenando {nan_before_step2[col]} NaNs en la columna '{col}' con el valor lógico 0.")
             X[col] = X[col].fillna(0)
             nan_after_step2[col] = X[col].isnull().sum()
        else:
             print(f"   Columna '{col}' no contiene NaNs, no se necesita rellenar.")
             nan_after_step2[col] = 0
# Imprimir resumen del paso 2
print("   Resumen Paso 2 (NaNs):")
for col in problem_cols:
     if col in nan_before_step2:
        print(f"     '{col}': Antes={nan_before_step2[col]}, Después={nan_after_step2[col]}")


Paso 2: Imputando NaNs específicos con 0...
   Rellenando 2732 NaNs en la columna 'Flow Bytes/s' con el valor lógico 0.
   Rellenando 2732 NaNs en la columna 'Flow Packets/s' con el valor lógico 0.
   Resumen Paso 2 (NaNs):
     'Flow Bytes/s': Antes=2732, Después=0
     'Flow Packets/s': Antes=2732, Después=0


In [12]:
# 3. Imputación General con la mediana para CUALQUIER OTRO NaN restante en columnas NUMÉRICAS
print("\nPaso 3: Imputando cualquier otro NaN numérico con la mediana...")
numeric_cols = X.select_dtypes(include=np.number).columns
nan_before_step3 = X[numeric_cols].isnull().sum().sum() - sum(nan_after_step2.get(col, 0) for col in problem_cols if col in X.columns) # Subtract NaNs already handled

if nan_before_step3 > 0:
    print(f"   Calculando medianas para columnas con {nan_before_step3} NaNs restantes...")
    # Calculate medians only for columns that still have NaNs after step 2
    cols_with_nan = X[numeric_cols].isnull().any()
    cols_to_fill = cols_with_nan[cols_with_nan].index
    if not cols_to_fill.empty:
        medians = X[cols_to_fill].median()
        # print(f"   Medianas calculadas: {medians.to_dict()}") # Opcional: ver medianas
        X.fillna(medians, inplace=True) # Rellena solo donde X tiene NaN, usando las medianas correspondientes
        print(f"   {nan_before_step3} NaNs numéricos rellenados con medianas.")
    else:
        print("   No se encontraron columnas adicionales con NaNs para rellenar.")

else:
    print("   No se encontraron otros NaNs numéricos para rellenar con mediana.")


Paso 3: Imputando cualquier otro NaN numérico con la mediana...
   No se encontraron otros NaNs numéricos para rellenar con mediana.


In [13]:
# 4. Verificación final de NaNs
print("\nPaso 4: Verificando NaNs restantes...")
nan_remaining = X.isnull().sum().sum()
if nan_remaining > 0:
    print(f"ADVERTENCIA: Todavía quedan {nan_remaining} valores NaN después de la limpieza.")
    print("   Columnas con NaNs restantes:")
    print(X.isnull().sum()[X.isnull().sum() > 0])
else:
    print("¡Verificación de NaN exitosa! No quedan valores NaN en X.")

# 5. Saneamiento de valores negativos
print("\nPaso 5: Reemplazando valores negativos específicos con 0...")
negative_check_cols = ["Init_Win_bytes_forward", "Init_Win_bytes_backward"]
for col in negative_check_cols:
    if col in X.columns:
        neg_count_before = (X[col] < 0).sum()
        if neg_count_before > 0:
            X.loc[X[col] < 0, col] = 0
            neg_count_after = (X[col] < 0).sum()
            print(f"   En '{col}', {neg_count_before} valores negativos reemplazados. Negativos restantes: {neg_count_after}")
        else:
            print(f"   En '{col}', no se encontraron valores negativos.")
    else:
        print(f"   Columna '{col}' no encontrada para chequeo de negativos.")

print("\n--- Limpieza de datos completada --- ")


Paso 4: Verificando NaNs restantes...
¡Verificación de NaN exitosa! No quedan valores NaN en X.

Paso 5: Reemplazando valores negativos específicos con 0...
   En 'Init_Win_bytes_forward', 919278 valores negativos reemplazados. Negativos restantes: 0
   En 'Init_Win_bytes_backward', 1339179 valores negativos reemplazados. Negativos restantes: 0

--- Limpieza de datos completada --- 


Verificación Post-Limpieza (Opcional pero recomendado)

In [14]:
print("\n--- Verificación Post-Limpieza ---")
print("Verificando de nuevo si hay NaNs en X:")
final_nan_check = X.isnull().sum().sum()
print(f"   Total NaNs en X: {final_nan_check}")
if final_nan_check > 0:
     print(X.isnull().sum()[X.isnull().sum() > 0])


print("\nVerificando de nuevo si hay Inf en X:")
final_inf_check = np.isinf(X.select_dtypes(include=np.number).values).sum()
print(f"   Total Infinitos en X: {final_inf_check}")


--- Verificación Post-Limpieza ---
Verificando de nuevo si hay NaNs en X:
   Total NaNs en X: 0

Verificando de nuevo si hay Inf en X:
   Total Infinitos en X: 0


Recombinación y Guardado del Dataset Limpio

In [15]:
if y is not None:
    # Asegurarse de que los índices coincidan si se han eliminado filas (aunque aquí no se eliminaron)
    X_aligned = X.reindex(y.index) if X.index.equals(y.index) else X
    df_cleaned = pd.concat([X_aligned, y.rename('y')], axis=1)
    print("   Características (X) y objetivo (y) recombinados.")
    # Opcionalmente, puedes volver a añadir 'Attack_Label' si lo necesitas
    # if 'Attack_Label' in df.columns:
    #    df_cleaned = pd.concat([X_aligned, y.rename('y'), df['Attack_Label'].reindex(y.index)], axis=1)
else:
    df_cleaned = X.copy()
    print("   Solo características (X) presentes.")

print("\nDimensiones del dataset limpio:", df_cleaned.shape)

   Características (X) y objetivo (y) recombinados.

Dimensiones del dataset limpio: (2660377, 20)


In [16]:
# Guardar el dataframe limpio
print(f"Guardando dataset limpio en: {CLEANED_DATASET_PATH}")
try:
    df_cleaned.to_csv(CLEANED_DATASET_PATH, index=False)
    print("✅ ¡Dataset limpio guardado exitosamente!")
except Exception as e:
    print(f"❌ Error al guardar el dataset limpio: {e}")

Guardando dataset limpio en: ../data/processed/ddos_findef.csv
✅ ¡Dataset limpio guardado exitosamente!
